1b. Dada una imagen trazar una ventana de proyección y proyectar la imagen

In [42]:
import cv2
import numpy as np

# --- Variables globales ---
src_points = []
dst_points = []
phase = 0   # 0 = seleccionando src, 1 = seleccionando dst
image = None
clone = None

def ordenar_cuatro_puntos(pts):
    """
    Ordena 4 puntos en el orden:
    Top-Left, Top-Right, Bottom-Right, Bottom-Left
    """
    pts = np.array(pts, dtype=np.float32)
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)

    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]
    return np.array([tl, tr, br, bl], dtype=np.float32)

def mouse_callback(event, x, y, flags, param):
    global src_points, dst_points, phase, image, clone

    if event == cv2.EVENT_LBUTTONDOWN:
        if phase == 0 and len(src_points) < 4:
            src_points.append([x, y])
            cv2.circle(image, (x, y), 5, (0, 255, 0), -1)  # verde
            cv2.imshow("Proyeccion interactiva", image)

            if len(src_points) == 4:
                print("Selecciona ahora los 4 puntos destino (rojo).")
                phase = 1

        elif phase == 1 and len(dst_points) < 4:
            dst_points.append([x, y])
            cv2.circle(image, (x, y), 5, (0, 0, 255), -1)  # rojo
            cv2.imshow("Proyeccion interactiva", image)

            if len(dst_points) == 4:
                print("Calculando proyección...")
                proyectar()

def proyectar():
    global src_points, dst_points, clone, image

    # Reordenar automáticamente para evitar cruces
    src = ordenar_cuatro_puntos(src_points)
    dst = ordenar_cuatro_puntos(dst_points)

    # Homografía
    H, _ = cv2.findHomography(src, dst)

    h, w = clone.shape[:2]
    warped = cv2.warpPerspective(clone, H, (w, h))

    # --- Crear máscara de la zona destino ---
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.fillPoly(mask, [np.int32(dst)], 255)

    # --- Fondo negro, la original desaparece ---
    result = np.zeros_like(clone)
    cv2.copyTo(warped, mask, result)

    # Mostrar resultado en la misma ventana
    cv2.imshow("Proyeccion interactiva", result)
    cv2.imwrite("output/resultado.png", result)
    print("Resultado guardado en 'output/resultado.png'")

def main():
    global image, clone

    ruta = "images/cats.png"   # cambia la ruta de tu imagen
    clone = cv2.imread(ruta)
    if clone is None:
        raise FileNotFoundError("No se pudo cargar la imagen.")
    image = clone.copy()

    cv2.namedWindow("Proyeccion interactiva")
    cv2.setMouseCallback("Proyeccion interactiva", mouse_callback)
    cv2.imshow("Proyeccion interactiva", image)

    print("Haz click en 4 puntos fuente (verde), luego en 4 puntos destino (rojo).")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Haz click en 4 puntos fuente (verde), luego en 4 puntos destino (rojo).
Selecciona ahora los 4 puntos destino (rojo).
Calculando proyección...
Resultado guardado en 'output/resultado.png'
